In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Set the paths
train_dir = './dataset/train'
val_dir = './dataset/val'
test_dir = './dataset/test'

# ImageDataGenerator for data augmentation
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Flow images from the directories
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=8,
    class_mode='categorical'
)


base_model = VGG16(weights='imagenet', include_top=False,
                   input_shape=(224, 224, 3))

# Freeze the convolutional base
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(7, activation='softmax')(x)  # 7 categories

# Create the model
model = Model(inputs=base_model.input, outputs=x)


model.compile(
    optimizer=Adam(learning_rate=0.0001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=40,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)


test_loss, test_acc = model.evaluate(
    test_generator, steps=test_generator.samples // test_generator.batch_size)
print(f'Test accuracy: {test_acc}')


2024-09-19 21:25:24.586843: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-19 21:25:24.681549: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-19 21:25:24.713035: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-19 21:25:24.723092: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-19 21:25:24.780461: I tensorflow/core/platform/cpu_feature_guar

Found 1232 images belonging to 7 classes.
Found 310 images belonging to 7 classes.
Found 522 images belonging to 7 classes.


I0000 00:00:1726761326.458529    4668 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726761326.545023    4668 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726761326.545189    4668 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1726761326.546286    4668 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

Epoch 1/40


/home/shashank-jangde/Data/major/.venv/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()
I0000 00:00:1726761328.380289    4990 service.cc:146] XLA service 0x70281400d3d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1726761328.380310    4990 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 3050 Ti Laptop GPU, Compute Capability 8.6
2024-09-19 21:25:28.403361: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-19 21:25:28.538600: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuD

...

154/154 ━━━━━━━━━━━━━━━━━━━━ 12s 73ms/step - accuracy: 0.7769 - loss: 0.6688 - val_accuracy: 0.8553 - val_loss: 0.5248
Epoch 38/40
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 260us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8333 - val_loss: 0.2340
Epoch 39/40


2024-09-19 21:29:09.011377: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14576801579602680849
2024-09-19 21:29:09.011414: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 3680059314447041040
2024-09-19 21:29:09.048316: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14576801579602680849
2024-09-19 21:29:09.048353: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 3680059314447041040


154/154 ━━━━━━━━━━━━━━━━━━━━ 12s 75ms/step - accuracy: 0.7808 - loss: 0.6292 - val_accuracy: 0.8158 - val_loss: 0.5384
Epoch 40/40
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 261us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.6667 - val_loss: 0.8946
 1/65 ━━━━━━━━━━━━━━━━━━━━ 5s 79ms/step - accuracy: 0.6250 - loss: 0.8970

2024-09-19 21:29:20.998465: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14576801579602680849
2024-09-19 21:29:20.998495: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 3680059314447041040
2024-09-19 21:29:21.035545: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14576801579602680849
2024-09-19 21:29:21.035573: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 3680059314447041040


65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 48ms/step - accuracy: 0.7868 - loss: 0.6491
Test accuracy: 0.7942307591438293


In [2]:
# Unfreeze the top layers of the model
for layer in base_model.layers[-4:]:  # Unfreeze the last 4 layers, adjust as needed
    layer.trainable = True

# Re-compile the model with a lower learning rate
model.compile(
    optimizer=Adam(learning_rate=1e-5),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Continue training
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=20,
    validation_data=val_generator,
    validation_steps=val_generator.samples // val_generator.batch_size
)


Epoch 1/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 16s 85ms/step - accuracy: 0.7606 - loss: 0.6468 - val_accuracy: 0.8783 - val_loss: 0.4219
Epoch 2/20


2024-09-19 21:29:40.667005: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14576801579602680849
2024-09-19 21:29:40.667033: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 3680059314447041040


154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8333 - val_loss: 0.3460
Epoch 3/20


2024-09-19 21:29:41.052314: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14576801579602680849
2024-09-19 21:29:41.052341: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 3680059314447041040


154/154 ━━━━━━━━━━━━━━━━━━━━ 13s 83ms/step - accuracy: 0.8122 - loss: 0.5515 - val_accuracy: 0.8750 - val_loss: 0.3999
Epoch 4/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 311us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8333 - val_loss: 0.3622
Epoch 5/20


...

2024-09-19 21:31:11.258849: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14576801579602680849
2024-09-19 21:31:11.258902: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 3680059314447041040


154/154 ━━━━━━━━━━━━━━━━━━━━ 13s 79ms/step - accuracy: 0.8570 - loss: 0.4062 - val_accuracy: 0.9211 - val_loss: 0.2995
Epoch 18/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 299us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 0.8333 - val_loss: 0.4805
Epoch 19/20


2024-09-19 21:31:23.854889: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14576801579602680849
2024-09-19 21:31:23.854926: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 3680059314447041040
2024-09-19 21:31:23.897452: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14576801579602680849
2024-09-19 21:31:23.897493: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 3680059314447041040


154/154 ━━━━━━━━━━━━━━━━━━━━ 13s 80ms/step - accuracy: 0.9036 - loss: 0.3402 - val_accuracy: 0.9243 - val_loss: 0.3052
Epoch 20/20
154/154 ━━━━━━━━━━━━━━━━━━━━ 0s 252us/step - accuracy: 0.0000e+00 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.1014


2024-09-19 21:31:36.795786: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 14576801579602680849
2024-09-19 21:31:36.795812: I tensorflow/core/framework/local_rendezvous.cc:423] Local rendezvous recv item cancelled. Key hash: 3680059314447041040


In [3]:
model.save("train.keras")